# **Pré-Processador**

---



---



## Interface Pré-Processador


In [ ]:
from abc import ABC, abstractmethod

class Pre_Processador(ABC):
    _palavras_excluir = ['o', 'ao', 'aos' ,'os', 'a', 'as', 'e', 'um', 'uma', 'ele', 'ela', 'eles', 'elas',
              'do', 'da', 'dos', 'das', 'de', 'no', 'na', 'nos', 'nas', 'pelo',
              'pela', 'pelos', 'pelas', 'num', 'numa', 'nuns', 'numas', 'dum',
              'duma', 'duns', 'dumas']
    @abstractmethod
    def pre_processar(self, documento_valor, documento_classe):
      return valor, classe, num_palavras, num_classes
    @abstractmethod
    def retornar_classes(self):
        return classes

## Pré-Processador Frequência

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer


class Pre_Processador_Frequencia(Pre_Processador):
    def __init__(self, documento_valor, documento_classe):
        self.labelencoder = LabelEncoder() #transforma classes em numeros
        self.labelencoder.fit(documento_classe) # treinar para indexar rotulos categoricos em numeros
        print(self.labelencoder.classes_)
        #mindf sera frequencia minima para entrar no vocabulario  e o stopwords sera palavras para retirar dos registros
        self.vetorizar = CountVectorizer (min_df = 2, lowercase = True, strip_accents = 'unicode', stop_words = Pre_Processador._palavras_excluir)  
        self.vetorizar.fit(documento_valor)   #treinar nosso vocabulario para depois vetorizar
        print(self.vetorizar.vocabulary_)
        
    def pre_processar(self, documento_valor, documento_classe):
        classe = self.labelencoder.transform(documento_classe) # transforma rotulos categoricos em numeros
        print(classe)
        valor = self.vetorizar.transform(documento_valor).toarray() # vetoriza o valor de nosso documento de acordo com nosso vocabulario e passa para array
        print(valor)
        num_palavras=len(self.vetorizar.vocabulary_)  #tamanho de nosso vocabulario
        print(num_palavras)
        num_classes=len(self.labelencoder.classes_)   #quantidade de classes
        print(num_classes)
        return valor, classe, num_palavras, num_classes

    def retornar_classes(self):
        return self.labelencoder.classes_

## Pré-Processador Sequência

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer


class Pre_Processador_Sequencia(Pre_Processador):
    def __init__(self, documento_valor, documento_classe):
        self.labelencoder = LabelEncoder()
        self.labelencoder.fit(documento_classe)
        print(self.labelencoder.classes_)
        self.vetorizar = Tokenizer()                    #tokenizador para vetorizar
        valor = self.valor_filtrado(documento_valor)
        self.vetorizar.fit_on_texts(valor)    #treina a lista de vocabulario
        valor = self.vetorizar.texts_to_sequences(valor)   #vetoriza o texto para uma lista sequencial de inteiros com o valor filtrado 
        self.vetor_max = max([len(z) for z in valor])  #tamanho maximo de um vetor de sequencia palavras
        print(self.vetor_max)

    def valor_filtrado(self, documento_valor):
        valor = documento_valor
        valor_filtrado = []
        for x in valor:   #tirar palavras excluir
            for palavra in Pre_Processador._palavras_excluir:
                x = x.replace(palavra.center(1), '')    #retira as palavras_excluir com espaco(conectores)
            valor_filtrado.append(x)    
        return valor_filtrado

    def pre_processar(self, documento_valor, documento_classe):
        classe = self.labelencoder.transform(documento_classe)  # transforma rotulos categoricos em numeros
        print(classe)
        valor = self.valor_filtrado(documento_valor)            
        valor = self.vetorizar.texts_to_sequences(valor)    #vetoriza nosso conteudo para uma lista sequencial de inteiros com o valor ja filtrado        
        valor = pad_sequences(valor, maxlen=self.vetor_max, padding='post')  #vetorizar completando os vazios depois
        print(valor)
        num_palavras = len(self.vetorizar.word_docs)   #numero de palavras
        print(num_palavras)
        num_classes=len(self.labelencoder.classes_)    #numero de classes
        return  valor, classe, num_palavras, num_classes, self.vetor_max 

    def retornar_classes(self):
        return self.labelencoder.classes_

# **Classificador**

---



---



## Interface Classificador

In [ ]:
from abc import ABC, abstractmethod


class Classificador(ABC):
    @abstractmethod
    def dividirTeste(self, valor, classe):
        pass
    @abstractmethod
    def treinar(self):
        pass
    @abstractmethod
    def desempenho(self):
        pass
    @abstractmethod
    def classificar(self, array_x):
        return array_y
    @abstractmethod
    def abrir_modelo(self, modelo):
        pass    

## Naive Bayes

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB


class Classificador_Naive(Classificador):
    def __init__(self, valor, classe):
        self.naiveBayes = GaussianNB()
        self.valor_treino, self.valor_teste, self.classe_treino, self.classe_teste = train_test_split(valor, classe, test_size=0.33, random_state = 40)
    
    def dividirTeste(self, valor, classe):
        # ira dividir nossa base de testes e treino 33%, random_state sera o embaralhamento dos dados antes de dividir
        self.valor_treino, self.valor_teste, self.classe_treino, self.classe_teste = train_test_split(valor, classe, test_size=0.33, random_state = 40)
 
    def treinar(self):
        self.naiveBayes.fit(self.valor_treino, self.classe_treino)
        print("Classificador Treinado")

    def desempenho(self):
        desempenho=self.naiveBayes.score(self.valor_teste, self.classe_teste)
        print("O desempenho é de :",desempenho)
    
    def classificar(self, array_x):
        array_y = self.naiveBayes.predict_proba(array_x)
        return array_y

    def abrir_modelo(self, modelo):
        pass  
    

## Rede Neural MLP Simples

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint


#rede neural keras
#criar apenas 2 camadas ocultas

class Classificador_Rede_Neural_Simples(Classificador):
    def __init__(self, valor, classe, num_palavras, num_classe):
        self.qtdNeuronio = (num_palavras + num_classe) / 2  #para calcular o numero de neuronios de cada camada media entre num atributos e num classes
        self.rn = Sequential()
        #self.modelo_pesos = save_pesos
        self.valor_treino, self.valor_teste, self.classe_treino, self.classe_teste = train_test_split(valor, classe, test_size=0.33, random_state = None)
        self.rn.add(Dense(units = self.qtdNeuronio, activation = 'relu', input_dim = self.valor_treino.shape[1]))
        self.rn.add(Dense(units = self.qtdNeuronio, activation = 'relu'))
        self.rn.add(Dense(units = num_classe, activation = 'softmax'))    # softmax usado em problemas de classificacao para saber a classe vencedora
        self.rn.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
        
    def dividirTeste(self, valor, classe):
        # ira dividir nossa base de testes e treino 33%, random_state sera o embaralhamento dos dados antes de dividir
        self.valor_treino, self.valor_teste, self.classe_treino, self.classe_teste = train_test_split(valor, classe, test_size=0.33, random_state = None)
        #self.valor_treino, self.valor_teste, self.classe_treino, self.classe_teste = train_test_split(valor, classe, test_size=0.33, random_state = 40)

    def treinar(self):
        self.modelo_pesos = ModelCheckpoint('weights.best_dense.hdf5', monitor='val_acc', save_best_only=True, mode='max')
        #treinar com 50 epocas, fara o model checkpoint para salvar e carregar o ajuste de pesos, batch size é o tamanho do exemplo de treino de cada epoca
        #self.rn.fit(self.valor_treino, self.classe_treino, validation_data = (self.valor_teste, self.classe_teste), batch_size=256, epochs = 50, callbacks = [self.modelo_pesos])
        self.rn.fit(self.valor_treino, self.classe_treino, validation_data = (self.valor_teste, self.classe_teste), batch_size=40, epochs = 50 , callbacks = [self.modelo_pesos])
        #rn.load_weights('weights.best_dense.hdf5')   #abrir melhor performance pesos treinado
        print("Classificador Treinado")

    def desempenho(self):
        desempenho=self.rn.evaluate(self.valor_teste, self.classe_teste)
        print(desempenho)
    
    def classificar(self, array_x):
        array_y = self.rn.predict_proba(array_x)
        return array_y
    
    def abrir_modelo(self, modelo):
        self.rn.load_weights(modelo) 


## Rede Neural Densa 

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, GlobalMaxPooling1D, Dropout
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint


class Classificador_Rede_Neural_Densa(Classificador):
    def __init__(self, valor, classe, num_palavras, num_classe, vetor_max):
        self.rn = Sequential()
        #self.modelo_pesos = save_pesos
        self.valor_treino, self.valor_teste, self.classe_treino, self.classe_teste = train_test_split(valor, classe, test_size=0.33, random_state = None)
        self.rn.add(Embedding(input_dim=(num_palavras + 1), output_dim=80, input_length=vetor_max, trainable = True))     # mapear palavras que tem proximidade semantica (input dim - palavras unicas, output dim - vetor saida, input length - comprimento vetor)
        self.rn.add(GlobalMaxPooling1D())   # diminuir e resumir informacoes
        self.rn.add(Dropout(0.3))     #DropOut usado para prevenir overffiting, desativa 30% das sinapses aleatoriamente
        #self.rn.add(Dense(units = 7, activation = 'softmax'))
        self.rn.add(Dense(units = 4, activation = 'softmax'))
        self.rn.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
        
    def dividirTeste(self, valor, classe):
        # ira dividir nossa base de testes e treino 33%, random_state sera o embaralhamento dos dados antes de dividir
        self.valor_treino, self.valor_teste, self.classe_treino, self.classe_teste = train_test_split(valor, classe, test_size=0.33, random_state = None)

    def treinar(self):
        self.modelo_pesos = ModelCheckpoint('weights.best_dense.hdf5', monitor='val_acc', save_best_only=True, mode='max')
        #treinar com 60 epocas, fara o model checkpoint para salvar e carregar o ajuste de pesos, batch size é o tamanho do exemplo de treino de cada epoca
        self.rn.fit(self.valor_treino, self.classe_treino, validation_data = (self.valor_teste, self.classe_teste), batch_size = 5, epochs = 50, callbacks = [self.modelo_pesos])
        #rn.load_weights('weights.best_dense.hdf5')   #abrir melhor performance pesos treinado
        print("Classificador Treinado")

    def desempenho(self):
        desempenho=self.rn.evaluate(self.valor_teste, self.classe_teste)
        print(desempenho)
    
    def classificar(self, array_x):
        array_y = self.rn.predict_proba(array_x)
        return array_y
    
    def abrir_modelo(self, modelo):
        self.rn.load_weights(modelo) 

# **Motor**

---



---



In [ ]:
import pandas as pd

class Motor():
    def __init__(self, endereco_arquivo, tipo_classificador):
        self.endereco = endereco_arquivo
        dataset = pd.read_csv(self.endereco)
        print(dataset)
        #documento_valor, documento_classe = dataset.iloc[:,2], dataset.iloc[:,1] # seleciona todos as linhas ':' e as colunas 1 e 2
        documento_valor, documento_classe = dataset.iloc[:,1], dataset.iloc[:,0] # seleciona todos as linhas ':' e as colunas 0 e 1
        if tipo_classificador == 'Naive Bayes':
            self.pre_processador = Pre_Processador_Frequencia(documento_valor, documento_classe)
            valor, classe, num_palavras, num_classes = self.pre_processador.pre_processar(documento_valor, documento_classe)
            self.classificador = Classificador_Naive(valor, classe)
            self.classificador.treinar()
            self.classificador.desempenho()
        elif tipo_classificador == 'Rede Neural Simples':
            self.pre_processador = Pre_Processador_Frequencia(documento_valor, documento_classe)
            valor, classe, num_palavras, num_classes = self.pre_processador.pre_processar(documento_valor, documento_classe)
            self.classificador = Classificador_Rede_Neural_Simples(valor, classe, num_palavras, num_classes)
            self.classificador.treinar()
            self.classificador.desempenho()
        elif tipo_classificador == 'Rede Neural Densa':
            self.pre_processador = Pre_Processador_Sequencia(documento_valor, documento_classe)
            valor, classe, num_palavras, num_classes, vetor_max = self.pre_processador.pre_processar(documento_valor, documento_classe)
            self.classificador = Classificador_Rede_Neural_Densa(valor, classe, num_palavras, num_classes, vetor_max)
            self.classificador.treinar()
            self.classificador.desempenho()

    def classificar(self, endereco_arquivo):
        dataset = pd.read_csv(endereco_arquivo)
        print(dataset)
        #documento_valor, documento_classe = dataset.iloc[:,2], dataset.iloc[:,1] # seleciona todos as linhas ':' e as colunas 1 e 2
        documento_valor, documento_classe = dataset.iloc[:,1], dataset.iloc[:,0] # seleciona todos as linhas ':' e as colunas 0 e 1
        if self.pre_processador.__class__.__name__ == 'Pre_Processador_Frequencia':
            valor, classe, num_palavras, num_classes = self.pre_processador.pre_processar(documento_valor, documento_classe)
        elif self.pre_processador.__class__.__name__ == 'Pre_Processador_Sequencia':
            valor, classe, num_palavras, num_classes, vetor_max = self.pre_processador.pre_processar(documento_valor, documento_classe)
        else:
          print("Processador não encontrado")
        array_y = self.classificador.classificar(valor)   # retorno matriz probabilidades de cada classe
        classes_y = self.pre_processador.retornar_classes()   # retorno vetor das classes
        return classes_y, array_y

# **Teste**

---



---



In [ ]:
#carregar arquivos a partir do computador
from google.colab import files
uploaded = files.upload()
arquivo = list(uploaded.keys())[0]

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
print(arquivo)

Saving local1.csv to local1 (1).csv
User uploaded file "local1.csv" with length 257103 bytes
local1.csv


In [ ]:
#carregar arquivos a partir do computador

arquivo_treino = '/content/local_4.csv'
arquivo_teste = '/content/local_teste.csv'
motor = Motor(arquivo_treino, 'Rede Neural Densa')
x, y = motor.classificar(arquivo_teste)

print("Resultado classificacao",x,y)
print(x)
for i in range(len(y)):
    for j in range(len(x)):
        print("{:.2f}".format(y[i][j]))